# 1. Install Dependencies and Setup

In [ ]:
!pip install tensorflow tensorflow-gpu opencv-python matplotlib.pyplot

In [ ]:
!pip install tensorflow-gpu

In [ ]:
!pip list

In [1]:
import tensorflow as tf
import os

In [2]:
# Avoid OOM errors by setting GPU Memory Consumption Growth
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus: 
    tf.config.experimental.set_memory_growth(gpu, True)

In [3]:
tf.config.list_physical_devices('GPU')

[]

# 2. Remove dodgy images

In [4]:
import cv2
import imghdr

In [5]:
data_dir = 'cats and dogs-train'

In [ ]:
image_exts = ['jpeg','jpg', 'bmp', 'png']

In [ ]:
# Loop through each class in the dataset directory
for image_class in os.listdir(data_dir):  
    # Loop through each image in the current class directory
    for image in os.listdir(os.path.join(data_dir, image_class)):  
        # Construct the full file path for the image
        image_path = os.path.join(data_dir, image_class, image)  
        try:
            # Attempt to read the image using OpenCV
            img = cv2.imread(image_path)  
            # Check the image's file extension/type using imghdr
            tip = imghdr.what(image_path)  
            
            # If the image's extension/type is not in the list of valid extensions
            if tip not in image_exts:  
                # Print a message indicating the image is invalid
                print('Image not in ext list {}'.format(image_path))  
                # Remove the invalid image from the directory
                os.remove(image_path)  
        except Exception as e:
            # If an exception occurs (e.g., corrupted image), print the issue
            print('Issue with image {}'.format(image_path))  
            # os.remove(image_path)


# 3. Load Data

In [6]:
import numpy as np
from matplotlib import pyplot as plt

In [7]:
data = tf.keras.utils.image_dataset_from_directory(data_dir)

Found 557 files belonging to 2 classes.


In [8]:
data_iterator = data.as_numpy_iterator()

In [9]:
batch = data_iterator.next()

In [10]:
batch[0].shape

(32, 256, 256, 3)

In [ ]:
fig, ax = plt.subplots(ncols=4, figsize=(20,20))
for idx, img in enumerate(batch[0][:4]):
    ax[idx].imshow(img.astype(int))
    ax[idx].title.set_text(batch[1][idx])

# 4. Scale Data

In [11]:
data = data.map(lambda x,y: (x/255, y))

In [12]:
data.as_numpy_iterator().next()

(array([[[[0.98039216, 0.972549  , 0.9764706 ],
          [0.98039216, 0.972549  , 0.9764706 ],
          [0.9843137 , 0.9764706 , 0.98039216],
          ...,
          [1.        , 1.        , 1.        ],
          [1.        , 1.        , 1.        ],
          [1.        , 1.        , 1.        ]],
 
         [[0.98039216, 0.972549  , 0.9764706 ],
          [0.98039216, 0.972549  , 0.9764706 ],
          [0.9843137 , 0.9764706 , 0.98039216],
          ...,
          [1.        , 1.        , 1.        ],
          [1.        , 1.        , 1.        ],
          [1.        , 1.        , 1.        ]],
 
         [[0.98039216, 0.972549  , 0.9764706 ],
          [0.98039216, 0.972549  , 0.9764706 ],
          [0.9843137 , 0.9764706 , 0.98039216],
          ...,
          [1.        , 1.        , 1.        ],
          [1.        , 1.        , 1.        ],
          [1.        , 1.        , 1.        ]],
 
         ...,
 
         [[0.6968605 , 0.63874084, 0.63693404],
          [0.60387

# 5. Split Data

In [13]:
train_size = int(len(data)*.8)
val_size = int(len(data)*.2)+1

In [14]:
train_size

14

In [15]:
train = data.take(train_size)
val = data.skip(train_size).take(val_size)

# 6. Build Deep Learning Model

In [16]:
train

<TakeDataset element_spec=(TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout

In [16]:
model = Sequential()

In [17]:
model.add(Conv2D(16, (3,3), 1, activation='relu', input_shape=(256,256,3)))
model.add(MaxPooling2D())
model.add(Conv2D(32, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())
model.add(Conv2D(16, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [18]:
model.compile('adam', loss=tf.losses.BinaryCrossentropy(), metrics=['accuracy'])

In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 254, 254, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 127, 127, 16)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 125, 125, 32)      4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 62, 62, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 60, 60, 16)        4624      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 30, 30, 16)       0

# 7. Train

In [20]:
logdir='logs'

In [21]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

In [22]:
hist = model.fit(train, epochs=20, validation_data=val, callbacks=[tensorboard_callback])

Epoch 1/20
14/14 [==============================] - 23s 1s/step - loss: 1.0080 - accuracy: 0.4576 - val_loss: 0.6979 - val_accuracy: 0.5229
Epoch 2/20
14/14 [==============================] - 17s 1s/step - loss: 0.6952 - accuracy: 0.4710 - val_loss: 0.6936 - val_accuracy: 0.4404
Epoch 3/20
14/14 [==============================] - 16s 1s/step - loss: 0.6879 - accuracy: 0.5156 - val_loss: 0.6791 - val_accuracy: 0.5596
Epoch 4/20
14/14 [==============================] - 17s 1s/step - loss: 0.6787 - accuracy: 0.5536 - val_loss: 0.6561 - val_accuracy: 0.7431
Epoch 5/20
14/14 [==============================] - 18s 1s/step - loss: 0.6696 - accuracy: 0.6808 - val_loss: 0.6290 - val_accuracy: 0.7248
Epoch 6/20
14/14 [==============================] - 18s 1s/step - loss: 0.6138 - accuracy: 0.7344 - val_loss: 0.5659 - val_accuracy: 0.7156
Epoch 7/20
14/14 [==============================] - 16s 1s/step - loss: 0.5365 - accuracy: 0.7478 - val_loss: 0.5051 - val_accuracy: 0.7156
Epoch 8/20
14/14 [==

# 8. Plot Performance

In [ ]:
fig = plt.figure()
plt.plot(hist.history['loss'], color='teal', label='loss')
plt.plot(hist.history['val_loss'], color='orange', label='val_loss')
fig.suptitle('Loss', fontsize=20)
plt.legend(loc="upper left")
plt.show()

In [ ]:
fig = plt.figure()
plt.plot(hist.history['accuracy'], color='teal', label='accuracy')
plt.plot(hist.history['val_accuracy'], color='orange', label='val_accuracy')
fig.suptitle('Accuracy', fontsize=20)
plt.legend(loc="upper left")
plt.show()

# 9. Evaluate

In [ ]:
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy

In [ ]:
pre = Precision()
re = Recall()
acc = BinaryAccuracy()

In [ ]:
for batch in test.as_numpy_iterator(): 
    X, y = batch
    yhat = model.predict(X)
    pre.update_state(y, yhat)
    re.update_state(y, yhat)
    acc.update_state(y, yhat)

In [ ]:
print(pre.result(), re.result(), acc.result())

# 10. Test

In [ ]:
import cv2

In [ ]:
img = cv2.imread('pic_11.jpg')
plt.imshow(img)
plt.show()

In [ ]:
resize = tf.image.resize(img, (256,256))
plt.imshow(resize.numpy().astype(int))
plt.show()

In [ ]:
yhat = model.predict(np.expand_dims(resize/255, 0))

In [ ]:
yhat

In [ ]:
if yhat > 0.5: 
    print(f'Predicted class is Dog')
else:
    print(f'Predicted class is Cat')

# 11. Save the Model

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
model.save(os.path.join('models','imageclassifier.h5'))

In [ ]:
new_model = load_model('imageclassifier.h5')

In [ ]:
new_model.predict(np.expand_dims(resize/255, 0))